In [1]:
%cd ../
%matplotlib inline

/home/eli/AnacondaProjects/HTFATorch


In [2]:
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils

import csv
import glob
import logging
import os
import re

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

db_file = 'data/synthetic_data.db'
mask_file = '/data/eli/synthetic_data/nifti_files/wholebrain.nii.gz'

OVERWRITE = True

In [4]:
TASK_1_SUBJECTS = list(range(1, 11))
TASK_2_SUBJECTS = list(range(11, 21))

In [5]:
existed = os.path.isfile(db_file)
db = niidb.FMriActivationsDb(db_file, mask=mask_file)

if not existed or OVERWRITE:
    data_files = glob.glob('/data/eli/synthetic_data/nifti_files/subject_*.nii')
    regex = re.compile("subject_([0-9]{1,2}).nii")
    for data_file in data_files:
        m = regex.search(data_file)
        [subject] = m.groups()
        subject = int(subject)
        logging.info("Recording blocks in %s", data_file)
        
        synthetic_block = niidb.FMriActivationBlock(zscore=False)
        synthetic_block.task = '1' if subject in TASK_1_SUBJECTS else '2'
        synthetic_block.filename = data_file
        synthetic_block.subject = subject
        synthetic_block.start_time = 0
        synthetic_block.end_time = None
        db.upsert(synthetic_block)
        logging.info('Subject %d performing task %s', subject, synthetic_block.task)

10/03/2018 15:19:00 Recording blocks in /data/eli/synthetic_data/nifti_files/subject_4.nii
10/03/2018 15:19:00 Subject 4 performing task 1
10/03/2018 15:19:00 Recording blocks in /data/eli/synthetic_data/nifti_files/subject_20.nii
10/03/2018 15:19:00 Subject 20 performing task 2
10/03/2018 15:19:00 Recording blocks in /data/eli/synthetic_data/nifti_files/subject_5.nii
10/03/2018 15:19:00 Subject 5 performing task 1
10/03/2018 15:19:00 Recording blocks in /data/eli/synthetic_data/nifti_files/subject_17.nii
10/03/2018 15:19:00 Subject 17 performing task 2
10/03/2018 15:19:00 Recording blocks in /data/eli/synthetic_data/nifti_files/subject_3.nii
10/03/2018 15:19:00 Subject 3 performing task 1
10/03/2018 15:19:00 Recording blocks in /data/eli/synthetic_data/nifti_files/subject_1.nii
10/03/2018 15:19:00 Subject 1 performing task 1
10/03/2018 15:19:00 Recording blocks in /data/eli/synthetic_data/nifti_files/subject_9.nii
10/03/2018 15:19:00 Subject 9 performing task 1
10/03/2018 15:19:00 Rec

In [6]:
logging.info('Finished building NiiDb out of synthetic dataset')

10/03/2018 15:19:01 Finished building NiiDb out of synthetic dataset
